In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd

from scipy.optimize import curve_fit
from scipy.constants import k, e

In [ ]:
# Retrieve file path
data_dir = "../data"
csv_file = os.path.join(data_dir, "2c_led.csv")

# Define csv column name remap
csv_column_remap = {
    "out": "V_out (V)",
    "A0": "V_a0 (V)",
    "A1": "V_in - A1 (V)",
    "A2": "Ground (V)",
    "A0-A1": "Voltage Drop Across R (A0 - A1) (V)",
    "A1-A2": "Voltage Drop Across LED (A1 - Ground) (V)",
    "current (mA)": "Current Through R and LED (mA)",
}


# Read csv to dataframe
df = pd.read_csv(csv_file)

# Rename columns for easier identification
df = df.rename(columns=csv_column_remap)

df.head()

In [ ]:
# Fit the I-V characteristic of the LED: I = I_s * (exp(V/(nV_T)) - 1)
def led_iv_curve(V, I_s, n):
    V_T = k * 300 / e  # Thermal voltage at room temp (T=300K)
    return I_s * (np.exp(V / (n * V_T)) - 1)

# Use measured voltage across LED and current through LED
V = df["Voltage Drop Across LED (A1 - Ground) (V)"]
I = df["Current Through R and LED (mA)"] * 1e-3  # Convert mA to A

# Initial parameter guesses: I_s ~ 1e-12 A, n ~ 2
p0 = [9.24e-10, 7.06]

# Fit the curve
popt, pcov = curve_fit(led_iv_curve, V, I, p0=p0, maxfev=10000)
I_s_fit, n_fit = popt

print(f"Fitted saturation current I_s: {I_s_fit:.2e} A")
print(f"Fitted ideality factor n: {n_fit:.2f}")

# Plot the data and the fit
plt.figure(figsize=(8,5))
plt.scatter(V, I*1e3, label="Measured Data", color="blue")
V_fit = np.linspace(V.min(), V.max(), 200)
I_fit = led_iv_curve(V_fit, *popt)
plt.plot(V_fit, I_fit*1e3, label="Fit", color="red")
plt.xlabel("Voltage Across LED (V)")
plt.ylabel("Current Through LED (mA)")
plt.title("LED I-V Characteristic and Fit")
plt.legend()
plt.grid(True)
plt.show()